In [ ]:
thresh = np.quantile(data["Rainfall"], np.linspace(0.5, 0.98, 50))
mean_distances = np.zeros_like(thresh)
xis = np.zeros_like(thresh)
sigmas = np.zeros_like(thresh)
num_excesses = np.zeros_like(thresh, dtype=int)
seed = 0
rng = np.random.default_rng(seed)

k = 100
m = 500

for i, u in enumerate(thresh):

    extremes = data["Rainfall"][data["Rainfall"] > u].values
    excess = extremes - u
    num_excesses[i] = len(excess)

    if num_excesses[i] < 10:
        continue

    mle_0 = excess.mean()

    model = ThresholdExcessModel(
        data=extremes, threshold=u, num_years=num_years
    )
    model.fit()

    xis[i] = model.theta[0]
    sigmas[i] = model.theta[1]

    distances = np.zeros(k)

    try:
        for j in range(k):

            x = rng.choice(excess, size=num_excesses[i], replace=True)
            mle = x.mean()

            model = ThresholdExcessModel(
                data=x, threshold=u, num_years=num_years
            )
            model.fit()
            quants = model.eval_quantile(np.arange(1, m+1) / (m + 1))

            distances[j] = np.sum(np.abs(np.quantile(x, np.arange(1, m+1) / (m + 1)) - quants)) / m
        mean_distances[i] = distances.mean()
    except:
        mean_distances[i] = np.nan


In [ ]:
min_indx = np.argmin(mean_distances[~np.isnan(mean_distances)])
chosen_threshold = thresh[min_indx]
chosen_threshold